In [5]:
# 필요한 패키지 import
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [6]:
# 엑셀 저장 함수 정의
def save_excel(df):
    excel = pd.ExcelWriter('../data/elecarCharge.xlsx')
    df.to_excel(excel, '.', index=False)
    excel.save()

In [8]:
# API key 가져오기 (보안을 위해 텍스트 파일로 따로 저장해둠)
api_key_file = open('../data/API_key', 'r')
api_key = api_key_file.readline()
api_key_file.close()

In [9]:
# 전역 변수 선언 (전체 레코드가 저장될 리스트)
my_list = []

In [10]:
# HTTP 통신으로 데이터 가져오기
# 전체 리스트에 저장
for page_n in range(1,3):
    url = 'http://open.ev.or.kr:8080/openapi/services/EvCharger/getChargerInfo?serviceKey=' \
          + api_key + '&pageSize=' + str(120000) +'&pageNo=' + str(page_n)

    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    data = soup.find_all('item')

    for item in data:
        my_list.append(pd.DataFrame({'충전소명':[item.find('statnm').text], '충전소ID':[item.find('statid').text],
                                       '충전기ID':[item.find('chgerid').text], '충전기터입':[item.find('chgertype').text],
                                       '주소':[item.find('addr').text], '위도':[item.find('lat').text],
                                       '경도':[item.find('lng').text], '이용가능시간':[item.find('usetime').text],
                                       '기관ID':[item.find('busiid').text], '기관명':[item.find('businm').text],
                                       '기관연락처':[item.find('busicall').text], '충전기상태':[item.find('stat').text],
                                       '갱신일시':[item.find('statupddt').text], '충전량':[item.find('powertype').text]}))

In [11]:
# 전체 데이터프레임으로 병합
df = pd.concat(my_list)
# 인덱스 재정렬
df.reset_index(drop=True, inplace=True)

In [12]:
# 전체 데이터프레임 확인
print(df)

                   충전소명     충전소ID 충전기ID 충전기터입  \
0              종묘 공영주차장  ME000001    01    03   
1             세종로 공영주차장  ME000002    01    06   
2            그랜드앰배서더 서울  ME000003    01    06   
3            한강진역 공영주차장  ME000004    01    03   
4      마장동사무소 앞(공중전화부스)  ME000006    01    06   
...                 ...       ...   ...   ...   
17758          부강자동차공업사  PW011279    01    02   
17759          부강자동차공업사  PW011279    02    02   
17760               온수장  PW011292    01    02   
17761             드라마오션  PW011949    01    02   
17762             드라마오션  PW011949    02    02   

                                           주소               위도  \
0           서울특별시 종로구 종로 157, 지하주차장 4층 하층 T구역        37.571076   
1      서울특별시 종로구 세종대로 189, 지하주차장 4층 D구역 계단실 앞        37.573611   
2                   서울특별시 중구 동호로 287, 대형버스주차장        37.559352   
3               서울특별시 용산구 한남동 산10-84, 지상실외주차장        37.540085   
4                           서울특별시 성동구 마장동 808       37.5660935   
...            

In [13]:
# 전체 데이터프레임 엑셀로 저장
save_excel(df)